# Templatized notebook for running CB-Geo MPM TAPIS job

## Install DesignSafe API (dapi)

In [13]:
!pip uninstall dapi --yes
!pip install -e ../

Found existing installation: dapi 1.0.0
Uninstalling dapi-1.0.0:
  Successfully uninstalled dapi-1.0.0
Obtaining file:///Users/krishna/dev/DesignSafe/dapi
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for dapi (pyproject.toml) ... done
  Created wheel for dapi: filename=dapi-1.0.0-py3-none-any.whl size=3825 sha256=f9fd4761dd2e940e7cd0f96795280478e27aacfbc14275ffb57c0ee44f9b129c
  Stored in directory: /private/var/folders/w8/xz590jyd7r36zmxcspgzj3z40000gn/T/pip-ephem-wheel-cache-d8f5rlch/wheels/98/df/91/ed70fe2dca11c3c6e5b6e8e6eef18c373a119d095037f892a3
Successfully built dapi

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip


In [ ]:
import os

# Import only DSClient and exceptions needed at top level
from dapi import (
    DSClient,
    SubmittedJob,
    interpret_job_status,  # Import new function
    AppDiscoveryError,
    FileOperationError,
    JobSubmissionError,
    JobMonitorError,
    # Optionally import status constants if you want to check against them explicitly
    STATUS_TIMEOUT,
    STATUS_UNKNOWN,
    TAPIS_TERMINAL_STATES,
)
import json
from datetime import datetime
from dataclasses import asdict
import pandas as pd
import tqdm as notebook_tqdm

In [4]:
try:
    print("Initializing DSClient...")
    ds = DSClient()
    print("DSClient initialized.")
except Exception as e:
    print(f"Initialization failed: {e}")
    raise SystemExit("Stopping notebook due to client initialization failure.")

Initializing DSClient...
Authentication successful.
DatabaseAccessor initialized. Connections will be created on first access.
DSClient initialized.


In [6]:
ds_path: str = "/MyData/mpm-benchmarks/2d/uniaxial_stress/"
input_filename: str = "mpm.json"
max_job_minutes: int = 10
# queue: str = "skx" # Example override - only if needed and valid
tacc_allocation: str = "BCS20003"
app_id_to_use = "mpm"  # Or "mpm-s3" - use the correct ID verified on DesignSafe

In [8]:
try:
    input_uri = ds.files.translate_path_to_uri(ds_path)
    print(f"Input Directory Tapis URI: {input_uri}")
except Exception as e:
    print(f"Error translating path '{ds_path}': {e}")
    raise SystemExit("Stopping notebook due to path translation error.")

Translated '/MyData/mpm-benchmarks/2d/uniaxial_stress/' to 'tapis://designsafe.storage.default/kks32/mpm-benchmarks/2d/uniaxial_stress/' using t.username
Input Directory Tapis URI: tapis://designsafe.storage.default/kks32/mpm-benchmarks/2d/uniaxial_stress/


In [9]:
try:
    print("\nGenerating job request dictionary...")
    job_dict = ds.jobs.generate_request(
        app_id=app_id_to_use,
        input_dir_uri=input_uri,
        script_filename=input_filename,
        max_minutes=max_job_minutes,
        allocation=tacc_allocation,
    )
    print("\n--- Generated Job Request Dictionary ---")
    print(json.dumps(job_dict, indent=2, default=str))
    print("---------------------------------------")
except (AppDiscoveryError, ValueError, JobSubmissionError) as e:
    print(f"Error generating job request: {e}")
    raise SystemExit("Stopping notebook due to job request generation error.")
except Exception as e:
    print(f"An unexpected error occurred during job request generation: {e}")
    raise SystemExit("Stopping notebook due to unexpected generation error.")


Generating job request dictionary...
Generating job request for app 'mpm'...
Using App Details: mpm v1.1.0
Placing script 'mpm.json' in appArgs: 'Input Script'
Adding allocation: BCS20003
Job request dictionary generated successfully.

--- Generated Job Request Dictionary ---
{
  "name": "mpm-20250427_182648",
  "appId": "mpm",
  "appVersion": "1.1.0",
  "description": "Material Point Method (MPM) is a particle based method that represents the material as a collection of material points, and their deformations are determined by Newton\u2019s laws of motion.",
  "execSystemId": "frontera",
  "archiveSystemId": "frontera",
  "archiveOnAppError": true,
  "execSystemLogicalQueue": "normal",
  "nodeCount": 3,
  "coresPerNode": 56,
  "maxMinutes": 10,
  "memoryMB": 192000,
  "isMpi": false,
  "tags": [],
  "fileInputs": [
    {
      "name": "Input Directory",
      "sourceUrl": "tapis://designsafe.storage.default/kks32/mpm-benchmarks/2d/uniaxial_stress/",
      "autoMountLocal": true,
    

In [10]:
# At this point, the user can inspect and modify job_dict if needed.
# For example:
print("Modifying job request dictionary...")
job_dict["nodeCount"] = 1
job_dict["coresPerNode"] = 1
job_dict["execSystemLogicalQueue"] = "development"

print(json.dumps(job_dict, indent=2, default=str))

Modifying job request dictionary...
{
  "name": "mpm-20250427_182648",
  "appId": "mpm",
  "appVersion": "1.1.0",
  "description": "Material Point Method (MPM) is a particle based method that represents the material as a collection of material points, and their deformations are determined by Newton\u2019s laws of motion.",
  "execSystemId": "frontera",
  "archiveSystemId": "frontera",
  "archiveOnAppError": true,
  "execSystemLogicalQueue": "development",
  "nodeCount": 1,
  "coresPerNode": 1,
  "maxMinutes": 10,
  "memoryMB": 192000,
  "isMpi": false,
  "tags": [],
  "fileInputs": [
    {
      "name": "Input Directory",
      "sourceUrl": "tapis://designsafe.storage.default/kks32/mpm-benchmarks/2d/uniaxial_stress/",
      "autoMountLocal": true,
      "targetPath": "inputDirectory"
    }
  ],
  "parameterSet": {
    "appArgs": [
      {
        "name": "Input Script",
        "arg": "mpm.json"
      }
    ],
    "schedulerOptions": [
      {
        "name": "TACC Allocation",
       

In [11]:
if "job_dict" not in locals():
    print("Error: job_dict not found.")
    raise SystemExit("Stopping notebook.")
try:
    print("\nSubmitting the job request dictionary...")
    submitted_job = ds.jobs.submit_request(job_dict)
    print(f"Job Submitted Successfully!")
    print(f"Job UUID: {submitted_job.uuid}")
except JobSubmissionError as e:
    print(f"Job submission failed: {e}")
    print("\n--- Failed Job Request ---")
    print(json.dumps(job_dict, indent=2, default=str))
    print("--------------------------")
    raise SystemExit("Stopping notebook due to job submission error.")
except Exception as e:
    print(f"An unexpected error occurred during job submission: {e}")
    raise SystemExit("Stopping notebook due to unexpected submission error.")


Submitting the job request dictionary...

--- Submitting Tapis Job Request ---
{
  "name": "mpm-20250427_182648",
  "appId": "mpm",
  "appVersion": "1.1.0",
  "description": "Material Point Method (MPM) is a particle based method that represents the material as a collection of material points, and their deformations are determined by Newton\u2019s laws of motion.",
  "execSystemId": "frontera",
  "archiveSystemId": "frontera",
  "archiveOnAppError": true,
  "execSystemLogicalQueue": "development",
  "nodeCount": 1,
  "coresPerNode": 1,
  "maxMinutes": 10,
  "memoryMB": 192000,
  "isMpi": false,
  "tags": [],
  "fileInputs": [
    {
      "name": "Input Directory",
      "sourceUrl": "tapis://designsafe.storage.default/kks32/mpm-benchmarks/2d/uniaxial_stress/",
      "autoMountLocal": true,
      "targetPath": "inputDirectory"
    }
  ],
  "parameterSet": {
    "appArgs": [
      {
        "name": "Input Script",
        "arg": "mpm.json"
      }
    ],
    "schedulerOptions": [
      

In [13]:
if "submitted_job" not in locals():
    print("Error: submitted_job not found.")
    raise SystemExit("Stopping notebook.")

print(f"\nMonitoring job {submitted_job.uuid} with progress bar...")
# Call monitor - exceptions are handled inside now, returns status string
final_status = submitted_job.monitor(interval=15)  # Use 15s interval

print(f"\nJob {submitted_job.uuid} monitoring finished.")


Monitoring job 530fbb91-cc06-42cd-b765-9bfdd9197f76-007 with progress bar...


Waiting for job 530fbb91-cc06-42cd-b765-9bfdd9197f76-007 to start: 5checks [01:17, 15.46s/checks, Status: STAGING_JOB]   
Monitoring job 530fbb91-cc06-42cd-b765-9bfdd9197f76-007 (Status: RUNNING):   2%| | 1/40 [00:00<00:00

	Job 530fbb91-cc06-42cd-b765-9bfdd9197f76-007 Status Update: RUNNING


Monitoring job 530fbb91-cc06-42cd-b765-9bfdd9197f76-007 (Status: ARCHIVING):  15%|▏| 6/40 [01:17<08:

	Job 530fbb91-cc06-42cd-b765-9bfdd9197f76-007 Status Update: ARCHIVING


Monitoring job 530fbb91-cc06-42cd-b765-9bfdd9197f76-007 (Status: ARCHIVING): 100%|█| 40/40 [01:48<00


Job 530fbb91-cc06-42cd-b765-9bfdd9197f76-007 reached terminal state: FINISHED

Job 530fbb91-cc06-42cd-b765-9bfdd9197f76-007 monitoring finished.


In [14]:
print("\n--- Job Outcome ---")
ds.jobs.interpret_status(final_status, submitted_job.uuid)
print("-------------------")


--- Job Outcome ---
Job 530fbb91-cc06-42cd-b765-9bfdd9197f76-007 completed successfully.
-------------------


In [19]:
# Check against known good terminal states or the specific success state
if final_status in ["FINISHED", "FAILED"]:  # Or just: if final_status == "FINISHED":
    print(f"\nAttempting to display runtime summary...")
    try:
        submitted_job.print_runtime_summary(verbose=True)
    except Exception as e:
        print(f"Could not display runtime summary: {e}")
else:
    print(f"\nSkipping runtime summary because job ended with status: {final_status}.")


Attempting to display runtime summary...

Runtime Summary
---------------

Detailed Job History:
  Event: JOB_NEW_STATUS, Detail: PENDING, Time: 2025-04-27T23:28:05.993343Z
  Event: JOB_NEW_STATUS, Detail: PROCESSING_INPUTS, Time: 2025-04-27T23:28:06.983221Z
  Event: JOB_NEW_STATUS, Detail: STAGING_INPUTS, Time: 2025-04-27T23:28:10.936444Z
  Event: JOB_INPUT_TRANSACTION_ID, Detail: COMPLETED, Time: 2025-04-27T23:28:57.386229Z
  Event: JOB_NEW_STATUS, Detail: STAGING_JOB, Time: 2025-04-27T23:28:57.447678Z
  Event: JOB_NEW_STATUS, Detail: SUBMITTING_JOB, Time: 2025-04-27T23:29:30.292279Z
  Event: JOB_NEW_STATUS, Detail: QUEUED, Time: 2025-04-27T23:29:31.223575Z
  Event: JOB_NEW_STATUS, Detail: RUNNING, Time: 2025-04-27T23:29:44.363804Z
  Event: JOB_NEW_STATUS, Detail: ARCHIVING, Time: 2025-04-27T23:30:57.808106Z
  Event: JOB_ARCHIVE_TRANSACTION_ID, Detail: COMPLETED, Time: 2025-04-27T23:31:24.899366Z
  Event: JOB_NEW_STATUS, Detail: FINISHED, Time: 2025-04-27T23:31:31.847357Z

Summary:


In [18]:
# if final_status in TAPIS_TERMINAL_STATES and final_status != STATUS_UNKNOWN: # Check if it's a known end state
print(f"\nAttempting to access archive information...")
try:
    archive_uri = submitted_job.archive_uri
    if archive_uri:
        print(f"Job Archive Tapis URI: {archive_uri}")
        print("\nListing archive contents (root):")
        outputs = ds.files.list(archive_uri)
        if outputs:
            for item in outputs:
                print(
                    f"- {item.name} (Type: {item.type}, Size: {item.size} bytes, Modified: {item.lastModified})"
                )
        else:
            print("No files found in the archive root directory.")
    else:
        print("Archive URI not available for this job.")
except FileOperationError as e:
    print(f"Could not list archive files: {e}")
except Exception as e:
    print(f"An unexpected error occurred while accessing archive information: {e}")


Attempting to access archive information...
Job Archive Tapis URI: tapis://frontera/work2/05873/kks32/frontera/tapis-jobs-archive/2025-04-27Z/mpm-20250427_182648-530fbb91-cc06-42cd-b765-9bfdd9197f76-007

Listing archive contents (root):
Listing files in system 'frontera' at path 'work2/05873/kks32/frontera/tapis-jobs-archive/2025-04-27Z/mpm-20250427_182648-530fbb91-cc06-42cd-b765-9bfdd9197f76-007'...
Found 5 items.
- inputDirectory (Type: dir, Size: 4096 bytes, Modified: 2025-04-27T23:31:18Z)
- tapisjob.env (Type: file, Size: 1519 bytes, Modified: 2025-04-27T23:31:19Z)
- tapisjob.out (Type: file, Size: 5588 bytes, Modified: 2025-04-27T23:31:19Z)
- tapisjob.sh (Type: file, Size: 1208 bytes, Modified: 2025-04-27T23:31:19Z)
- tapisjob_app.sh (Type: file, Size: 189 bytes, Modified: 2025-04-27T23:31:11Z)


In [26]:
# Find all apps (less verbose)
all_apps = ds.apps.find("", verbose=False)
print(f"Found {len(all_apps)} total apps.")

Found 90 total apps.


In [27]:
# Find MPM apps specifically
mpm_apps = ds.apps.find("mpm", verbose=True)


Found 2 matching apps:
- mpm (Version: 1.1.0, Owner: wma_prtl)
- mpm-s3 (Version: 1.0, Owner: wma_prtl)



In [ ]:
# Get details for the specific MPM app we want to use
app_id_to_use = "opensees-express"
app_details = ds.apps.get_details(app_id_to_use, verbose=True)

if not app_details:
    raise SystemExit(
        f"Could not find details for app '{app_id_to_use}'. Please check the app ID."
    )
# Print the app details

print(f"App Description: {app_details}")


App Details:
  ID: opensees-express
  Version: latest
  Owner: wma_prtl
  Execution System: wma-exec-01
  Description: OpenSees-EXPRESS provides users with a sequential OpenSees interpreter. It is ideal to run small sequential scripts on DesignSafe resources freeing up your own machine.
App Description: 
containerImage: tapis://cloud.data/corral/tacc/aci/CEP/applications/v3/opensees/latest/OpenSees-EXPRESS/opensees_express.zip
created: 2025-02-20T18:41:03.661272Z
deleted: False
description: OpenSees-EXPRESS provides users with a sequential OpenSees interpreter. It is ideal to run small sequential scripts on DesignSafe resources freeing up your own machine.
enabled: True
id: opensees-express
isPublic: True
jobAttributes: 
archiveOnAppError: True
archiveSystemDir: /tmp/${JobOwner}/tapis-jobs-archive/${JobCreateDate}/${JobName}-${JobUUID}
archiveSystemId: cloud.data
cmdPrefix: None
coresPerNode: 1
description: None
dtnSystemInputDir: !tapis_not_set
dtnSystemOutputDir: !tapis_not_set
dyna